In [2]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp39-cp39-linux_x86_64.whl size=663060 sha256=f01087f36133ca66ae3f508e294bb7650a2b148d0570840f52e6f25dabecb688
  Stored in directory: /home/RRAM_HKU/.cache/pip/wheels/4c/79/a2/654387bb6950d38a2995edd69ef44cbaa1518e9c389cf81e6d
Successfully built pycuda


In [4]:
!pip install tensorrt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-10.0.1-py2.py3-none-any.whl size=16348 sha256=76799c4e5c3cbcfa240e0ce6470196fcac53962c5ce6db71829be5e6dd7251d7
  Stored in directory: /home/RRAM_HKU/.cache/pip/wheels/66/35/d9/b4cc85dc78d44db7878680839a709752259b4b1c90c2a8f309
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.0.1-py2.py3-none-any.whl size=17568 sha256=2e0ebc2aa2b4d5032e5030f1025892c651f80c48639f7044559f076bf11f5a30
  Stored in directory: /home/RRAM_HKU/.cache/pip/wheels/39/02/07/c74ab0730014857bb0c785fdd43ffd85e6af0a2c25a75b9954
Successfully built tensorrt tensorrt-cu12


In [5]:
import os
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
import cv2

In [10]:
# Set the path to the directory containing the images
image_path = "./JPEGImages"
# model_path = "./segformer_fp16_2.engine"
model_path = "./segformer.trt"
# model_path = "./darknet_fp16.engine"

# Set the input size expected by your TensorRT model
input_height = 640
input_width = 640
input_channels = 3

In [11]:
# Load the TensorRT model
TRT_LOGGER = trt.Logger(trt.Logger.INFO)
with open(model_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Create execution context
context = engine.create_execution_context()

# Get input and output names
input_name = engine.get_binding_name(0)
output_name = engine.get_binding_name(1)

[05/04/2024-12:58:22] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[05/04/2024-12:58:22] [TRT] [I] Loaded engine size: 25 MiB
[05/04/2024-12:58:22] [TRT] [E] 1: [runtime.cpp::parsePlan::455] Error Code 1: Serialization (Serialization assertion plan->header.magicTag == rt::kPLAN_MAGIC_TAG failed.Trying to load an engine created with incompatible serialization version. Check that the engine was not created using safety runtime, same OS was used and version compatibility parameters were set accordingly.)


AttributeError: 'NoneType' object has no attribute 'create_execution_context'

In [9]:
# Allocate device memory for input and output
input_shape = (1, input_channels, input_height, input_width)
output_shape = engine.get_binding_shape(1)
input_size = trt.volume(input_shape) * np.dtype(np.float32).itemsize
output_size = trt.volume(output_shape) * np.dtype(np.float32).itemsize
d_input = cuda.mem_alloc(input_size)
d_output = cuda.mem_alloc(output_size)


AttributeError: 'NoneType' object has no attribute 'get_binding_shape'

In [13]:

# Create a stream to run inference
stream = cuda.Stream()

In [14]:
# Get the list of image files in the directory
image_files = [f for f in os.listdir(image_path) if f.endswith((".jpg", ".jpeg", ".png"))]
num_images = len(image_files)

def preprocess_image(image_path, input_height, input_width):
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    
    # Resize the image to the desired input shape
    resized_image = cv2.resize(image, (input_width, input_height))
    
    # Convert the image from BGR to RGB color space
    rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    # Normalize the pixel values to the range [0, 1]
    normalized_image = rgb_image.astype(np.float32) / 255.0
    
    # Transpose the image to match the input shape (channels first)
    transposed_image = normalized_image.transpose((2, 0, 1))
    
    # Add batch dimension to the image
    batch_image = np.expand_dims(transposed_image, axis=0)
    
    return batch_image

In [15]:
# Perform inference on each image

total_time = 0
for image_file in image_files:
    # Preprocess the image
    image = preprocess_image(os.path.join(image_path, image_file), input_height, input_width)

    start_time = time.time()
    
    # Transfer input data to device
    cuda.memcpy_htod_async(d_input, image.ravel(), stream)
    
    # Run inference
    context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
    
    # Transfer output data from device
    output = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh_async(output, d_output, stream)
    print(output)
    print(d_output)
    
    # Synchronize the stream to ensure the inference is complete
    stream.synchronize()

    end_time = time.time()
    total_time = total_time + (end_time - start_time)
    
    # Postprocess the output (if needed)
    # ...

# end_time = time.time()
# total_time = end_time - start_time
inference_speed = num_images / total_time

print(f"Processed {num_images} images in {total_time:.2f} seconds")
print(f"Inference speed: {inference_speed:.2f} images per second")

[05/03/2024-23:12:27] [TRT] [E] 1: [genericReformat.cu::executeMemcpy::1455] Error Code 1: Cuda Runtime (invalid argument)


LogicError: cuMemcpyDtoHAsync failed: an illegal memory access was encountered